<a href="https://colab.research.google.com/github/gchit21/ML4/blob/main/3LayerCnn_BatchNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kaggle and wandb Setup**

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge


 79% 226M/285M [00:00<00:00, 601MB/s] 
100% 285M/285M [00:03<00:00, 90.2MB/s]
Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install wandb onnx -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 66.9 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gchit21 (gchit21-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# **Model**

In [ ]:
import os
import random

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Turn the pixel string into an numpy Array and then into an Image**

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

from PIL import Image

import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

class FERDataset(Dataset):
    def __init__(self, csv_file, usage, transform=None):
        self.data = pd.read_csv(csv_file)
        self.data = self.data[self.data[' Usage'] == usage]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = np.fromstring(self.data.iloc[idx][' pixels'], sep=' ', dtype=np.uint8).reshape(48, 48)
        image = Image.fromarray(pixels)
        label = int(self.data.iloc[idx]['emotion'])

        if self.transform:
            image = self.transform(image)

        return image, label


**Model**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ThreeLayerBatch(nn.Module):
    def __init__(self, kernels, kernel_size, classes=7, drop_threshold=0.2):
        super(ThreeLayerBatch, self).__init__()

        assert len(kernels) == 3, "kernels list must have exactly 3 values (one for each conv layer)."

        self.conv1 = nn.Conv2d(1, kernels[0], kernel_size=kernel_size, padding=1)
        self.bn1 = nn.BatchNorm2d(kernels[0])

        self.conv2 = nn.Conv2d(kernels[0], kernels[1], kernel_size=kernel_size, padding=1)
        self.bn2 = nn.BatchNorm2d(kernels[1])

        self.conv3 = nn.Conv2d(kernels[1], kernels[2], kernel_size=kernel_size, padding=1)
        self.bn3 = nn.BatchNorm2d(kernels[2])

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(drop_threshold)
        self.flatten = nn.Flatten()


        self.fc1 = nn.Linear(kernels[2] * 6 * 6, 128)
        self.fc2 = nn.Linear(128, classes)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))  # 48 -> 24
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))  # 24 -> 12
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))  # 12 -> 6

        x = self.dropout(x)
        x = self.flatten(x)
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        return x


**Training**

In [ ]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        example_ct=0
        batch_ct=0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            loss, outputs = train_batch(images,labels,model,optimizer,criterion)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss, example_ct, epoch)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
        wandb.log({
          "epoch": num_epochs,
          "train/loss": round(running_loss / len(train_loader),4),
          "train/accuracy": round(100 * correct / total,2)
        })

def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss,outputs


**Train Logging**

In [ ]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

**Testing and Test Logging**

In [ ]:
def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    wandb.log({"Test Accuracy": round(100 * correct / total, 2)})

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [ ]:
def make(config):

    transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),])

    # Make the data
    train =  FERDataset("icml_face_data.csv", usage="Training", transform=transform)
    val =  FERDataset("icml_face_data.csv", usage="PublicTest", transform=transforms.ToTensor())

    train_loader = DataLoader(train, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=config.batch_size, shuffle=False)

    # Make the model
    model = ThreeLayerBatch(config.kernels,config.kernel_size,config.classes,config.drop_threshold).to(device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)

    return model, train_loader, val_loader, criterion, optimizer

In [ ]:
def model_pipeline(hyperparameters, run_name):

    # tell wandb to get started
    with wandb.init(project="3-Layer_BatchNorm", name=run_name, config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, train_loader, val_loader, criterion, optimizer = make(config)
      print(model)

      # and use them to train the model
      train_model(model, train_loader, criterion, optimizer, device, config.epochs)

      # and test its final performance
      test_model(model, val_loader, device)

    return model

In [ ]:
number = 27

In [ ]:
config = dict(
    epochs=10,
    classes=7,
    kernels=[30, 45 ,70],
    kernel_size = 3,
    batch_size=125,
    learning_rate=0.0005,
    drop_threshold=0.2,
    dataset="icml_face_data",
    architecture="CNN_BatchNorm")

name ="run_" + str(number)
number+=1

model = model_pipeline(config,name)

ThreeLayerBatch(
  (conv1): Conv2d(1, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(30, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(45, 70, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2520, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
)
Loss after 03000 examples: 1.678
Loss after 06125 examples: 1.721
Loss after 09250 examples: 1.620
Loss after 12375 examples: 1.659
Loss after 15500 examples: 1.659
Loss after 18625 examp

wandb: WARNING Tried to log to step 3000 that is less than the current step 28001. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Loss after 06125 examples: 1.401


wandb: WARNING Tried to log to step 6125 that is less than the current step 28001. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Loss after 09250 examples: 1.542


...

wandb: WARNING Tried to log to step 24875 that is less than the current step 28009. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Loss after 28000 examples: 0.958
Epoch 10/10, Loss: 1.0629, Accuracy: 59.75%


wandb: WARNING Tried to log to step 28000 that is less than the current step 28009. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Test Accuracy: 56.84%


Test Accuracy,▁
epoch,▁▁▁▁▁▁▁▁██████████
loss,▇█▆▇▇▄▄▁▃
train/accuracy,▁▄▅▆▆▇▇▇██
train/loss,█▅▄▃▃▂▂▂▁▁
Test Accuracy,56.84
epoch,10
loss,1.47826
train/accuracy,59.75
train/loss,1.0629
